<a href="https://colab.research.google.com/github/MoeinDSP/Simulating-social-media-platforms-with-Large-Language-Models/blob/main/Simulating_Social_Media_using_LLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1- Install dependencies.**

In [6]:
import os
import openai
import networkx as nx
import random
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

**2- Client Configuration**

In [7]:
client = openai.OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key="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"
)


**3- Create Persona**

In [8]:
class Persona:
    def __init__(self, id, name, age, location, job, mbti, interests):
        self.id = id
        self.name = name
        self.age = age
        self.location = location
        self.job = job
        self.mbti = mbti
        self.interests = interests
        self.profile = self._build_profile()
        self.embedding = np.random.rand(512)  # Dummy vector to support similarity mode

    def _build_profile(self):
        return (
            f"Name: {self.name}\n"
            f"Age: {self.age}\n"
            f"Location: {self.location}\n"
            f"Job: {self.job}\n"
            f"MBTI: {self.mbti}\n"
            f"Interests: {', '.join(self.interests)}"
        )

    def update_attributes(self):
        self.age += 1
        if random.random() < 0.1:
            self.location = random.choice(["New York", "Berlin", "Tokyo", "Sydney", "Toronto"])
        if random.random() < 0.1:
            self.job = random.choice(["Engineer", "Designer", "Writer", "Artist", "Manager"])
        self.profile = self._build_profile()
        self.embedding = np.random.rand(512)  # regenerate dummy embedding

**4- Network Simulator**

In [9]:
class SocialNetworkSimulator:
    def __init__(self, model="google/gemma-3-1b-it:free"):
        self.model = model
        self.graph = nx.Graph()
        self.personas = []

    def add_persona(self, persona):
        self.personas.append(persona)
        self.graph.add_node(persona.id, persona=persona)

    def simulate_round(self, strategy="random", k=2):
        pairs = []
        n = len(self.personas)

        if strategy == "random":
            ids = list(range(n))
            random.shuffle(ids)
            for i in range(0, n - 1, 2):
                pairs.append((self.personas[ids[i]], self.personas[ids[i+1]]))
        elif strategy == "one-to-one":
            for i in range(n - 1):
                pairs.append((self.personas[i], self.personas[i+1]))
        elif strategy == "similarity":
            emb = np.stack([p.embedding for p in self.personas])
            sim = cosine_similarity(emb)
            for i, p in enumerate(self.personas):
                sims = sorted(enumerate(sim[i]), key=lambda x: x[1], reverse=True)
                for j, _ in sims[1:k+1]:
                    pairs.append((p, self.personas[j]))

        for sender, receiver in pairs:
            try:
                msg = self._chat(sender, receiver)
                reaction, comment = self._react_and_comment(receiver, msg)
                print(f"{sender.name} -> {receiver.name}: {msg}\nReaction: {reaction}, Comment: {comment}\n---")
                self.graph.add_edge(sender.id, receiver.id)
            except Exception as e:
                print(f"Communication error: {e}")

    def _chat(self, sender, receiver):
        messages = [
            {
                "role": "system",
                "content": f"You are {sender.name}, a {sender.age}yo {sender.job} from {sender.location}. "
                           f"Your MBTI is {sender.mbti}. You are interested in {', '.join(sender.interests)}."
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": f"Send a friendly message to {receiver.name}, a {receiver.age}yo {receiver.job} from {receiver.location}."
                    }
                ]
            }
        ]

        response = client.chat.completions.create(
            model=self.model,
            messages=messages,
            extra_headers={"HTTP-Referer": "", "X-Title": ""},
            extra_body={},
            temperature=0.7,
            max_tokens=150
        )
        return response.choices[0].message.content

    def _react_and_comment(self, persona, message):
        reaction = random.choice(["like", "love", "haha", "wow", "sad", "angry"])
        response = client.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": f"You are {persona.name}. React and comment."},
                {"role": "user", "content": [
                    {"type": "text", "text": message}
                ]}
            ],
            extra_headers={"HTTP-Referer": "", "X-Title": ""},
            extra_body={},
            temperature=0.7,
            max_tokens=100
        )
        return reaction, response.choices[0].message.content

    def compute_metrics(self):
        return {
            "Degree Centrality": nx.degree_centrality(self.graph),
            "Graph Density": nx.density(self.graph),
            "Clustering Coefficient": nx.clustering(self.graph)
        }


if __name__ == "__main__":
    sim = SocialNetworkSimulator()
    people = [
        Persona(1, "Alice", 30, "Berlin", "Engineer", "INTJ", ["tech", "music"]),
        Persona(2, "Bob", 25, "New York", "Designer", "ENFP", ["art", "travel"]),
        Persona(3, "Carol", 28, "Tokyo", "Writer", "INFJ", ["books", "coffee"]),
        Persona(4, "Dave", 35, "Sydney", "Manager", "ESTJ", ["sports", "cooking"])
    ]

    for p in people:
        sim.add_persona(p)

    for round in range(3):
        print(f"\n=== Round {round + 1} ===")
        sim.simulate_round(strategy="random")
        for p in sim.personas:
            p.update_attributes()

    metrics = sim.compute_metrics()
    for k, v in metrics.items():
        print(f"{k}: {v}")



=== Round 1 ===
Carol -> Alice: Please send a friendly message to Alice, a 30yo Engineer from Berlin:

Hey Alice! 👋

It’s Carol from Tokyo! I’ve been thinking about you and your amazing work in Berlin. I’m a writer, and I’m completely captivated by books – especially those that explore complex emotions and quiet moments. I also really love a good cup of coffee, you know? ☕️

Hope you’re having a fantastic week! 😊

Best,
Carol 

---

Would you like me to add anything specific? For example:

*   **A specific book recommendation?**
*   **A slightly more casual way to start the conversation?**
Reaction: wow, Comment:  should I add a book recommendation?
---
Dave -> Bob: Please send a friendly message to Bob, a 25yo Designer from New York:

Hey Bob!

Hope you're having a fantastic week so far! It’s great to connect with you – I’ve been keeping busy with a bit of a whirlwind of projects here in Sydney. 

I’m a 35-year-old Manager from Sydney, and I’m really into sports – especially [mention